# SUMMARIZING PAPERS CONTAINS TWO PARTS:-
- EXTRACTING TEXT FROM PDF/WORD FILE
- SUMMARIZING TEXT(EXTRACTIVE/ABSTRACTIVE)

# INSTALLING LIBRARIES

In [1]:
!pip install pdfplumber
!pip install python-docx
!pip install rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 69.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.6 MB/s eta 0:00:00a 0:00:01


## EXTRACTING TEXT FROM A WORD/PDF FILE
- We have two options for extracting text from pdf :- PyPDF2 and pdfplumber
- We are using pdfplumber library of Python rather than PyPDF2 because it retains some of the original formatting of the documents like table,bullet points,etc which are not retained by PyPDF2
- We will extract text from word file using docx library

In [2]:
import pdfplumber
from docx import Document
import os

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    print("Provided Document is PDF file\n\n")
    extracted_text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages):
                extracted_text += page.extract_text()
    except Exception as e:
        print(f"Error reading PDF file: {e}")
    return extracted_text

# Function to extract text from Word (.docx)
def extract_text_from_docx(docx_path):
    print("Provided Document is Word file\n\n")
    extracted_text = ""
    try:
        doc = Document(docx_path)
        extracted_text = '\n'.join([para.text for para in doc.paragraphs])
    except Exception as e:
        print(f"Error reading DOCX file: {e}")
    return extracted_text

# Function to handle both PDF and Word files
def extract_text_from_file(file_path):
    _, file_extension = os.path.splitext(file_path)
    
    if file_extension.lower() == '.pdf':
        return extract_text_from_pdf(file_path)
    elif file_extension.lower() == '.docx':
        return extract_text_from_docx(file_path)
    else:
        return f"Unsupported file format: {file_extension}"


# path
path = '/kaggle/input/papers/Bayesian Segnet.pdf'

# Extract text based on file type
extracted_text = extract_text_from_file(path)

# Print extracted text
# print("Extracted Text is: \n" )
# print(extracted_text)


tokens = len(extracted_text.split())
print(f'No of tokens in text: {tokens}')


Provided Document is PDF file


No of tokens in text: 3432


In [3]:
# Evaluation Metrix ROGUE

from rouge import Rouge


rouge = Rouge()



In [4]:
# Sample Text for Summary Generation on all models
extracted_text = """The Role of Renewable Energy in Sustainable Development
The pursuit of sustainable development has become one of the most pressing challenges of our time. As the global population continues to rise and the demand for energy increases, the reliance on fossil fuels poses significant environmental and economic threats. In this context, renewable energy sources have emerged as a viable alternative, offering a path toward sustainable and environmentally friendly energy solutions.

1. Understanding Renewable Energy

Renewable energy refers to energy derived from resources that are naturally replenished, such as sunlight, wind, rain, tides, waves, and geothermal heat. Unlike fossil fuels, which are finite and contribute to greenhouse gas emissions, renewable energy sources provide a cleaner and more sustainable energy supply. Solar power, for instance, harnesses sunlight using photovoltaic cells to generate electricity, while wind energy converts the kinetic energy of wind into power through turbines.

2. Environmental Benefits

The transition to renewable energy plays a crucial role in mitigating climate change. According to the Intergovernmental Panel on Climate Change (IPCC), adopting renewable energy could significantly reduce global greenhouse gas emissions, which are responsible for global warming and climate instability. By reducing our reliance on coal, oil, and natural gas, we can decrease air pollution, leading to improved public health outcomes and a reduction in healthcare costs associated with pollution-related illnesses.

3. Economic Opportunities

Investing in renewable energy not only benefits the environment but also creates substantial economic opportunities. The renewable energy sector has been a significant source of job creation, often outpacing traditional fossil fuel industries. In 2020, the International Renewable Energy Agency (IRENA) reported that the renewable energy sector employed over 11 million people worldwide, with jobs expected to increase as more countries commit to sustainable energy goals.

4. Energy Security and Independence

Renewable energy also enhances energy security and independence. By diversifying energy sources, countries can reduce their reliance on imported fuels, which can be subject to price volatility and geopolitical tensions. For instance, countries that invest in solar and wind energy can harness their local resources, leading to greater energy autonomy. This not only stabilizes energy prices but also enhances national security by reducing vulnerability to external energy supply disruptions.

5. Challenges and Considerations

Despite the numerous advantages, the transition to renewable energy is not without challenges. Issues such as the intermittent nature of renewable sources—like solar and wind—require the development of energy storage technologies and grid management solutions. Furthermore, the initial investment costs for renewable energy infrastructure can be high, although these costs have been decreasing rapidly due to technological advancements and economies of scale.

6. Conclusion

In conclusion, renewable energy represents a critical component of a sustainable future. By reducing greenhouse gas emissions, creating jobs, enhancing energy security, and providing clean power, renewable energy sources can significantly contribute to sustainable development. Policymakers, businesses, and individuals must work collaboratively to overcome existing challenges and accelerate the transition toward a greener, more sustainable energy landscape. As we move forward, embracing renewable energy will be essential in addressing the global challenges of climate change, economic inequality, and energy insecurity."""

## ABSTRACTIVE SUMMARIZATION

### It generates a summary by creating new sentences that capture the meaning of the original text, rather than extracting exact phrases or sentences.

### We can paraphrase the output text for better understanding 

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = "cpu"

paraphrase_tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

paraphrase_model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = paraphrase_tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = paraphrase_model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = paraphrase_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    

    return res

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

### Based on number of tokens in the extracted text we will choose the model which is best for that particular task.
- upto 1024 token :- BART(fine-tuned on CNN-Dailymail) AND PEGASUS(fine-tuned on CNN-Dailymail and various other)
- upto 4096 tokens :-  LONGT5(Pre-Trained on C4)
- upto 16,384 tokens :-Longformer (Encoder and Decoder) (fine-tuned on ArXiv and Pubmed)

In [6]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM 
from transformers import LEDForConditionalGeneration, LEDTokenizer
import torch
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer

In [7]:
summaries = {}

In [8]:
# BART Large
def summarize_with_bart_large(extracted_text):
    bart_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
    bart_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
    summarizer = pipeline("summarization", model=bart_model, tokenizer=bart_tokenizer, clean_up_tokenization_spaces=True)
    
    summarized_text = summarizer(extracted_text)
    
    
    
    paraphrased = paraphrase(summarized_text[0]["summary_text"])
    
    bart_paraphrased = ' '.join(paraphrased)
    
    
    summaries['BART'] = summarized_text[0]["summary_text"]
    
    bart_score = rouge.get_scores(extracted_text , summarized_text[0]["summary_text"])
    
    
    
    summaries['BART PARAPHRASED'] = bart_paraphrased
    
    summaries['BART ROUGE SCORES'] = bart_score
    
    
    print("Summary and Rouge scores and saved")
#     print(f'BART Summarized Text:- {summarized_text[0]["summary_text"]}\n\n')

In [9]:
# Pegasus Large
def summarize_with_pegasus_large(extracted_text):
    pegasus_tokenizer = AutoTokenizer.from_pretrained("google/pegasus-large")
    pegasus_model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-large")
    summarizer = pipeline("summarization", model=pegasus_model, tokenizer=pegasus_tokenizer, clean_up_tokenization_spaces=True)
    
    summarized_text = summarizer(extracted_text)
    
    
    paraphrased = paraphrase(summarized_text[0]["summary_text"])
    
    pegasus_paraphrased = ' '.join(paraphrased)
    
    pegasus_score = rouge.get_scores(extracted_text , summarized_text[0]["summary_text"])
    
    
    summaries['PEGASUS'] = summarized_text[0]["summary_text"]
    
    summaries['PEGASUS PARAPHRASED'] = pegasus_paraphrased
    
    summaries['PEGASUS ROUGE SCORES'] = pegasus_score
    
    print("Summary and Rouge score are saved")
#     print(f'PEGASUS Summarized Text:- {summarized_text[0]["summary_text"]}\n\n')

In [10]:
# LongT5
def summarize_with_longt5(extracted_text):
    longt5_tokenizer = AutoTokenizer.from_pretrained("google/long-t5-tglobal-base")
    longt5_model = AutoModelForSeq2SeqLM.from_pretrained("google/long-t5-tglobal-base")
    summarizer = pipeline("summarization", model=longt5_model, tokenizer=longt5_tokenizer, clean_up_tokenization_spaces=True)
    
    summarized_text = summarizer(extracted_text)
    
    
    paraphrased = paraphrase(summarized_text[0]["summary_text"])
    
    longt5_paraphrased = ' '.join(paraphrased)
    
    long_t5_score = rouge.get_scores(extracted_text , summarized_text[0]["summary_text"])
    
    
    summaries['LONGT5'] = summarized_text[0]["summary_text"]
    
    summaries['LONGT5 PARAPHRASED'] = longt5_paraphrased
    
    summaries['LONGT5 ROUGE SCORES'] = long_t5_score
    
    print("Summary and Rouge scores are saved")
#     print(f'LongT5 Summarized Text:- {summarized_text[0]["summary_text"]}\n\n')

In [11]:
def summarize_with_bigbird_pegasus(extracted_text):
        tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")

        # by default encoder-attention is `block_sparse` with num_random_blocks=3, block_size=64
        model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv")

        # decoder attention type can't be changed & will be "original_full"
        # you can change `attention_type` (encoder only) to full attention like this:
        model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", attention_type="original_full")

        # you can change `block_size` & `num_random_blocks` like this:
        model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", block_size=16, num_random_blocks=2)

        
        inputs = tokenizer(extracted_text, return_tensors='pt' , padding=False)
        prediction = model.generate(**inputs)
        prediction = tokenizer.batch_decode(prediction)
        
        
        paraphrased = paraphrase(prediction[0])
    
        bigbird_paraphrased = ' '.join(paraphrased)
    
    
        bigbird_score = rouge.get_scores(extracted_text , prediction[0])
    
        summaries['BIGBIRD'] = prediction[0]
        
        summaries['BIGBIRD PARAPHRASED'] = bigbird_paraphrased
        
        summaries['BIGBIRD ROGUE SCORES'] = bigbird_score
        
        print("Summary and Rouge scores are saved")
#         print(f"BIGBIRD Summarized Text: {prediction}")
        
        


In [12]:
# LED (Longformer Encoder-Decoder)
def summarize_with_led_large(extracted_text):
    tokenizer = LEDTokenizer.from_pretrained("allenai/led-large-16384-arxiv")

    input_ids = tokenizer(extracted_text, return_tensors="pt" , padding=False).input_ids
    global_attention_mask = torch.zeros_like(input_ids)
    # set global_attention_mask on first token
    global_attention_mask[:, 0] = 1

    model = LEDForConditionalGeneration.from_pretrained("allenai/led-large-16384-arxiv", return_dict_in_generate=True)

    sequences = model.generate(input_ids, global_attention_mask=global_attention_mask).sequences

    summary = tokenizer.batch_decode(sequences)
    
    paraphrased = paraphrase(summary[0])
    
    
    led_paraphrased = ' '.join(paraphrased)
    
    
    led_score = rouge.get_scores(extracted_text , summary[0])

    summaries["LED"] = summary[0]
    
    summaries['LED PARAPHRASED'] = led_paraphrased
    
    summaries['LED ROUGE SCORES'] = led_score
    
    print("Summary and rouge scores are saved")
    
#     print(f'LED LARGE Summarized Text: {summary}\n')

### Checking for each models if they can summarize

In [13]:
# BART
bart_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
tokenized_output = bart_tokenizer(extracted_text)
bart_tokens = len(tokenized_output[0])

print(f'BART TOKENIZED LENGTH: {bart_tokens}')
if bart_tokens <= 1024:
    print("Generating Summary...")
    summarize_with_bart_large(extracted_text)
else:
    print("BART can not be used here")

# Pegasus
pegasus_tokenizer = AutoTokenizer.from_pretrained("google/pegasus-large")
tokenized_output = pegasus_tokenizer(extracted_text)
pegasus_tokens = len(tokenized_output[0])

print(f'PEGASUS TOKENIZED LENGTH: {pegasus_tokens}')
if pegasus_tokens <= 1024:
    print("Generating Summary...")
    summarize_with_pegasus_large(extracted_text)
else:
    print("PEGASUS can not be used here")


# LongT5
longt5_tokenizer = AutoTokenizer.from_pretrained("google/long-t5-tglobal-base")
tokenized_output = longt5_tokenizer(extracted_text)
longt5_tokens = len(tokenized_output[0])

print(f'LONGT5 TOKENIZED LENGTH: {longt5_tokens}')
if longt5_tokens <= 4096:
    print("Generating Summary...")
    summarize_with_longt5(extracted_text)
else:
    print("LONGT5 can not be used here")

# BIGBIRD
bigbird_tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")
tokenized_output = bigbird_tokenizer(extracted_text, return_tensors="pt")
bigbird_tokens = tokenized_output['input_ids'].shape[1]


print(f'BIGBIRD TOKENIZED LENGTH: {bigbird_tokens}')
if bigbird_tokens <= 4096:
    print("Generating Summary...")
    summarize_with_bigbird_pegasus(extracted_text)
else:
    print("BIGBIRD cannot be used here")

# LED (Longformer Encoder-Decoder)
led_tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")
tokenized_output = led_tokenizer(extracted_text)
led_tokens = len(tokenized_output[0])

print(f'LED TOKENIZED LENGTH: {led_tokens}')
if led_tokens <= 16384:# LED can handle even longer sequences
    print("Generating Summary...")
    summarize_with_led_large(extracted_text)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

Input ids are automatically padded from 641 to 1024 to be a multiple of `config.attention_window`: 1024


Summary and rouge scores are saved


In [14]:
for model in summaries:
  print(model.upper())
  print(summaries[model])
  print("")

BART
Renewable energy sources provide a cleaner and more sustainable energy supply. By reducing our reliance on coal, oil, and natural gas, we can decrease air pollution. The renewable energy sector has been a significant source of job creation, often outpacing traditional fossil fuel industries. By diversifying energy sources, countries can reduce their reliance on imported fuels.

BART PARAPHRASED
The use of renewable energy sources can lead to a more sustainable and cleaner energy supply, as we can reduce air pollution by decreasing our dependence on coal, oil, and natural gas. The renewable sector has often created jobs that exceed the traditional fossil fuel industries, with diversifying energy resources helping countries reduce their dependency on imported fuels. A more sustainable supply of energy is available through renewable sources, which can reduce air pollution by reducing our dependence on coal, oil, and natural gas. The renewable energy sector has often provided employme

# EXTRACTIVE SUMMARIZATION

## It selects key sentences or phrases from the original text to form a concise summary without altering the original content.

In [15]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation
import networkx as nx

# Initialize stopwords
stop_words = list(stopwords.words('english'))

# Preprocess text
text = ' '.join(extracted_text.split())  # Remove extra whitespace
original_sentences = sent_tokenize(text)

# Clean sentences
cleaned_sentences = []
for sentence in original_sentences:
    # Convert to lowercase and remove punctuation
    cleaned = sentence.lower()
    cleaned = ''.join(char for char in cleaned if char not in punctuation)
    cleaned_sentences.append(cleaned)

# Calculate TF-IDF matrix. It will generate a matrix where each row represent a sentence and coloumn represent a term
tfidf = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix = tfidf.fit_transform(cleaned_sentences)

# Calculate similarity matrix between sentences
similarity_matrix = (tfidf_matrix * tfidf_matrix.T).toarray()

# Calculate TextRank scores
nx_graph = nx.from_numpy_array(similarity_matrix)
textrank_scores = nx.pagerank(nx_graph, alpha=0.85, max_iter=50)

# Calculate position scores which should be higher for starting and ending
num_sentences = len(original_sentences)
positions = np.arange(num_sentences)
middle = num_sentences // 2
position_scores = 1 - np.minimum(positions, num_sentences - 1 - positions) / middle

# Combine scores (0.7 weight for TextRank, 0.3 for position)
final_scores = {}
for idx in range(num_sentences):
    final_scores[idx] = 0.7 * textrank_scores[idx] + 0.3 * position_scores[idx]

# Select top sentences
min_sentences = 3
ratio = 0.3
num_sentences = max(min_sentences, int(len(original_sentences) * ratio))

# Sort sentences by score and get top ones
selected_indices = sorted(sorted(final_scores.items(), key=lambda x: x[1], reverse=True)[:num_sentences])

# Create summary maintaining original sentence order
summary_sentences = [original_sentences[idx] for idx, _ in selected_indices]
summary = ' '.join(summary_sentences)


extractive_score = rouge.get_scores(extracted_text , summary)

# Print results
print("Original Text Length:", len(text))
print("Summary Length:", len(summary))
print("\nSummary:\n", summary)


print(f'Rouge Scores are {extractive_score}')

Original Text Length: 3708
Summary Length: 1300

Summary:
 The Role of Renewable Energy in Sustainable Development The pursuit of sustainable development has become one of the most pressing challenges of our time. As the global population continues to rise and the demand for energy increases, the reliance on fossil fuels poses significant environmental and economic threats. In this context, renewable energy sources have emerged as a viable alternative, offering a path toward sustainable and environmentally friendly energy solutions. Understanding Renewable Energy Renewable energy refers to energy derived from resources that are naturally replenished, such as sunlight, wind, rain, tides, waves, and geothermal heat. Conclusion In conclusion, renewable energy represents a critical component of a sustainable future. By reducing greenhouse gas emissions, creating jobs, enhancing energy security, and providing clean power, renewable energy sources can significantly contribute to sustainable 